In [1]:
import numpy as np
import pandas as pd
import nltk
import sentence_transformers
from string import punctuation
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sentence_transformers import SentenceTransformer
from nltk import sent_tokenize, word_tokenize
from tqdm import tqdm
import torch.nn

c:\Users\Alex\scoop\apps\python\3.12.6\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import pandas as pd

splits = {'train': 'data/Synthetic-Persona-Chat_train.csv', 'validation': 'data/Synthetic-Persona-Chat_valid.csv', 'test': 'data/Synthetic-Persona-Chat_test.csv'}
df = pd.read_csv("hf://datasets/google/Synthetic-Persona-Chat/" + splits["train"])

In [3]:
personas = df['user 1 personas']+df['user 2 personas']

In [4]:
data = pd.DataFrame()
data['personas'] = personas
data['persona_id'] = range(len(data['personas']))
data['facts'] = range(len(data['personas']))

In [5]:
for index, row in data.iterrows():
    facts = row['personas'].split('.')
    new_facts = []
    for fact in facts:
      fact = fact.strip('\n')
      new_facts.append(fact)
    new_facts = new_facts[:-1]
    data['facts'][index] = new_facts

C:\Users\Alex\AppData\Local\Temp\ipykernel_49132\3111025079.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['facts'][index] = new_facts
C:\Users\Alex\AppData\Local\Temp\ipykernel_49132\3111025079.py:8: SettingWithCopyWarning: 
A val

In [6]:
data = data.explode('facts')
data.reset_index(drop=True, inplace=True)

In [7]:
data.head()

,personas,persona_id,facts
0,I am 32.\nI do not want a job.\nI play video g...,0,I am 32
1,I am 32.\nI do not want a job.\nI play video g...,0,I do not want a job
2,I am 32.\nI do not want a job.\nI play video g...,0,I play video games all day
3,I am 32.\nI do not want a job.\nI play video g...,0,I still live at home with my parents
4,I am 32.\nI do not want a job.\nI play video g...,0,My favorite drink is iced coffee


In [71]:
data.loc[data['persona_id']==259]

,personas,persona_id,facts,facts_embeddings,similar_personas,similar_facts
2338,I don t work.\nMy favorite drink is chocolate ...,259,I don t work,"[-0.054426435, -0.03574246, 0.051799424, -0.03...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{I eat junk food, I am a senior citizen, I am ..."
2339,I don t work.\nMy favorite drink is chocolate ...,259,My favorite drink is chocolate milk,"[-0.019659905, -0.05076409, 0.033380855, -0.01...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{I love to clean, I have a passion for god, I ..."
2340,I don t work.\nMy favorite drink is chocolate ...,259,I hate tacos,"[-0.021873755, -0.015252045, 0.03355251, 0.009...","{2, 4, 7, 8, 9, 10, 11, 12, 19, 21, 29, 30, 31...","{I love mexican food, I am lactose intolerant,..."
2341,I don t work.\nMy favorite drink is chocolate ...,259,I like to swim,"[-0.03467477, -0.049108848, 0.03294645, -0.005...","{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","{I am a senior citizen, I am known as the iron..."
2342,I don t work.\nMy favorite drink is chocolate ...,259,I like to arm wrestle,"[-0.07860201, -0.04728128, 0.024250073, -0.024...","{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{I love to clean, I have a passion for god, I ..."
2343,I don t work.\nMy favorite drink is chocolate ...,259,I am a cartographer,"[-0.02464995, -0.054552346, 0.052718308, -0.02...","{0, 2, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 1...","{I drive a jag, I drink beers and go to the cr..."
2344,I don t work.\nMy favorite drink is chocolate ...,259,I like to make bagels,"[-0.049451966, -0.0043938835, 0.028330773, -0....","{0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...","{I love to clean, I eat junk food, I am known ..."
2345,I don t work.\nMy favorite drink is chocolate ...,259,I like to paint outdoors,"[-0.0195053, -0.05657095, 0.058568057, -0.0060...","{1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15, 1...","{I love to clean, My hair is curly and dark, I..."
2346,I don t work.\nMy favorite drink is chocolate ...,259,I used to be a nurse,"[-0.042443093, -0.020305213, 0.049738903, -0.0...","{0, 1, 2, 3, 5, 6, 7, 8, 10, 12, 14, 15, 17, 2...",{I just accepted a job at the elementary schoo...


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer('BAAI/bge-small-en')
model = model.to(device)

In [9]:
facts_embeds = []
for i in tqdm(data['facts']):
   facts_embeds.append(model.encode(i))

100%|██████████| 80355/80355 [22:41<00:00, 59.00it/s]


In [10]:
import pickle
#with open('FINAL_facts_embeds.pkl', 'wb') as f:
 #   pickle.dump(facts_embeds, f)

In [11]:
#with open('FINAL_facts_embeds.pkl', 'rb') as handle:
  #  facts_embeds = pickle.load(handle)

In [12]:
data['facts_embeddings'] = facts_embeds

In [13]:
result = data[data['persona_id'] == 1230]

ids = result.index.tolist()

print(ids)

[11064, 11065, 11066, 11067, 11068, 11069, 11070, 11071, 11072]


In [14]:
data = data[:11073]

In [15]:
data.head()

,personas,persona_id,facts,facts_embeddings
0,I am 32.\nI do not want a job.\nI play video g...,0,I am 32,"[-0.043408405, -0.039896216, 0.004573693, -0.0..."
1,I am 32.\nI do not want a job.\nI play video g...,0,I do not want a job,"[-0.033704814, -0.019891072, 0.009261265, -0.0..."
2,I am 32.\nI do not want a job.\nI play video g...,0,I play video games all day,"[-0.02727933, 0.011558705, 0.029788923, -0.043..."
3,I am 32.\nI do not want a job.\nI play video g...,0,I still live at home with my parents,"[0.0048993924, 0.004853039, 0.04816079, -0.036..."
4,I am 32.\nI do not want a job.\nI play video g...,0,My favorite drink is iced coffee,"[-0.05509237, -0.028940937, 0.029442051, -0.00..."


In [16]:
def calculate_similarity_matrix(data):
    num_facts = len(data)
    similarity_matrix = np.zeros((num_facts, num_facts))
    cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    b =np.stack(data['facts_embeddings'].values)
    b_tensor = torch.tensor(b, dtype=torch.float32)

    for i in tqdm(range(num_facts)):
        for j in range(i+1, num_facts):
            similarity = cos(b_tensor[i], b_tensor[j])
            similarity_matrix[i][j] = similarity
    
    return similarity_matrix

In [47]:
fact_similarity_matrix = calculate_similarity_matrix(data)

100%|██████████| 11073/11073 [16:18<00:00, 11.31it/s] 


In [48]:
fact_similarity_matrix

array([[0.        , 0.83585423, 0.84408087, ..., 0.84687966, 0.80623251,
        0.79523307],
       [0.        , 0.        , 0.8053214 , ..., 0.8323437 , 0.81298524,
        0.81334984],
       [0.        , 0.        , 0.        , ..., 0.80795217, 0.76491904,
        0.76627737],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.81111515,
        0.89708626],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.85013247],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [49]:
fact_similarity_matrix.dump('FINAL_FINAL_fact_similarity_matrix.pkl')

In [51]:
n = fact_similarity_matrix.shape[0] 

for i in range(n):
    for j in range(i):
        fact_similarity_matrix[i, j] = fact_similarity_matrix[j, i]

In [52]:
fact_similarity_matrix

array([[0.        , 0.83585423, 0.84408087, ..., 0.84687966, 0.80623251,
        0.79523307],
       [0.83585423, 0.        , 0.8053214 , ..., 0.8323437 , 0.81298524,
        0.81334984],
       [0.84408087, 0.8053214 , 0.        , ..., 0.80795217, 0.76491904,
        0.76627737],
       ...,
       [0.84687966, 0.8323437 , 0.80795217, ..., 0.        , 0.81111515,
        0.89708626],
       [0.80623251, 0.81298524, 0.76491904, ..., 0.81111515, 0.        ,
        0.85013247],
       [0.79523307, 0.81334984, 0.76627737, ..., 0.89708626, 0.85013247,
        0.        ]])

In [53]:
data['similar_personas'] = [[] for _ in range(len(data))]
data['similar_facts'] = [[] for _ in range(len(data))]

In [142]:
for i in tqdm(range(len(data))):
    similar_indices = [j for j in range(len(data)) if fact_similarity_matrix[i][j] > 0.93 and i != j]
    data.at[i, 'similar_facts'] = list(set(data['facts'].iloc[similar_indices]))

100%|██████████| 11073/11073 [00:17<00:00, 622.04it/s]


In [143]:
for i in tqdm(range(len(data))):
    similar_indices = [j for j in range(len(data)) if fact_similarity_matrix[i][j] > 0.93 and i != j]
    data.at[i, 'similar_personas'] = list((set(data['persona_id'].iloc[similar_indices])))

100%|██████████| 11073/11073 [00:28<00:00, 382.70it/s]


In [144]:
data.head()

,personas,persona_id,facts,facts_embeddings,similar_personas,similar_facts
0,I am 32.\nI do not want a job.\nI play video g...,0,I am 32,"[-0.043408405, -0.039896216, 0.004573693, -0.0...","[1, 2, 649, 534, 422, 1080, 196, 197, 199, 75,...","[I am a 30 year old female, I m a 30 year old ..."
1,I am 32.\nI do not want a job.\nI play video g...,0,I do not want a job,"[-0.033704814, -0.019891072, 0.009261265, -0.0...","[1, 2, 354, 802, 648, 531, 504, 1080, 892, 862]","[I do not have a job, I do not like my job, I ..."
2,I am 32.\nI do not want a job.\nI play video g...,0,I play video games all day,"[-0.02727933, 0.011558705, 0.029788923, -0.043...","[1, 2, 641, 134, 135, 136, 137, 264, 902, 909,...","[I like video games, I enjoy video games, I pl..."
3,I am 32.\nI do not want a job.\nI play video g...,0,I still live at home with my parents,"[0.0048993924, 0.004853039, 0.04816079, -0.036...","[1, 2, 1028, 389, 1029, 263, 266, 522, 269, 14...","[I live at home with my dad and brother, I am ..."
4,I am 32.\nI do not want a job.\nI play video g...,0,My favorite drink is iced coffee,"[-0.05509237, -0.028940937, 0.029442051, -0.00...","[96, 514, 419, 421, 1029, 1159, 621, 1211, 496...","[I love coffee with cream, My favorite drink i..."


In [145]:
flattened_data = []

In [146]:
flattened_data = data.groupby('persona_id').agg({
    'personas': lambda x: list(x),
    'facts': lambda x: ', '.join(x),
    'facts_embeddings': lambda x: list(x),
    'similar_personas': lambda x: sorted(list(set(sum(x.tolist(), []))))
}).reset_index()

flattened_data.head()

,persona_id,personas,facts,facts_embeddings,similar_personas
0,0,[I am 32.\nI do not want a job.\nI play video ...,"I am 32, I do not want a job, I play video gam...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[1, 2, 14, 16, 57, 62, 75, 87, 96, 97, 100, 11..."
1,1,[I am 32.\nI play video games all day.\nI stil...,"I am 32, I play video games all day, I still l...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[0, 1, 2, 11, 14, 16, 27, 57, 62, 70, 75, 76, ..."
2,2,[I am 32.\nI play video games all day.\nI stil...,"I am 32, I play video games all day, I still l...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[0, 1, 14, 16, 25, 26, 27, 28, 35, 57, 62, 65,..."
3,3,[I write.\nI work at mcdonald s.\nI watch yout...,"I write, I work at mcdonald s, I watch youtube...","[[-0.052719686, -0.04074392, 0.03911773, -0.04...","[35, 39, 41, 45, 92, 95, 98, 99, 158, 159, 160..."
4,4,[I am bald.\nI like to swim.\nMy favorite drin...,"I am bald, I like to swim, My favorite drink i...","[[-0.024434138, -0.024031263, 0.04923339, -0.0...","[23, 35, 52, 69, 92, 96, 104, 105, 110, 115, 1..."


In [147]:
flattened_data['persona'] = data['personas']

pers_emb = flattened_data["facts_embeddings"].apply(lambda x: np.sum(list(x), axis=0)).reset_index()

flattened_data['persona_embeddings'] = pers_emb['facts_embeddings']

In [148]:
flattened_data.head()

,persona_id,personas,facts,facts_embeddings,similar_personas,persona,persona_embeddings
0,0,[I am 32.\nI do not want a job.\nI play video ...,"I am 32, I do not want a job, I play video gam...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[1, 2, 14, 16, 57, 62, 75, 87, 96, 97, 100, 11...",I am 32.\nI do not want a job.\nI play video g...,"[-0.27412432, -0.19816133, 0.18661429, -0.2283..."
1,1,[I am 32.\nI play video games all day.\nI stil...,"I am 32, I play video games all day, I still l...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[0, 1, 2, 11, 14, 16, 27, 57, 62, 70, 75, 76, ...",I am 32.\nI do not want a job.\nI play video g...,"[-0.27595556, -0.19046049, 0.36472562, -0.1551..."
2,2,[I am 32.\nI play video games all day.\nI stil...,"I am 32, I play video games all day, I still l...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[0, 1, 14, 16, 25, 26, 27, 28, 35, 57, 62, 65,...",I am 32.\nI do not want a job.\nI play video g...,"[-0.3358081, -0.24885227, 0.2707594, -0.333563..."
3,3,[I write.\nI work at mcdonald s.\nI watch yout...,"I write, I work at mcdonald s, I watch youtube...","[[-0.052719686, -0.04074392, 0.03911773, -0.04...","[35, 39, 41, 45, 92, 95, 98, 99, 158, 159, 160...",I am 32.\nI do not want a job.\nI play video g...,"[-0.34644818, -0.32542628, 0.3416062, -0.24792..."
4,4,[I am bald.\nI like to swim.\nMy favorite drin...,"I am bald, I like to swim, My favorite drink i...","[[-0.024434138, -0.024031263, 0.04923339, -0.0...","[23, 35, 52, 69, 92, 96, 104, 105, 110, 115, 1...",I am 32.\nI do not want a job.\nI play video g...,"[-0.2633983, -0.20231283, 0.28170836, -0.11812..."


In [149]:
def calculate_similarity_matrix(flattened_data):
    num_facts = len(flattened_data)
    similarity_matrix = np.zeros((num_facts, num_facts))
    cos = torch.nn.CosineSimilarity(dim=0, eps=1e-6)
    b =np.stack(flattened_data['persona_embeddings'].values)
    b_tensor = torch.tensor(b, dtype=torch.float32)

    for i in tqdm(range(num_facts)):
      for j in flattened_data['similar_personas'][i]:
        similarity = cos(b_tensor[i], b_tensor[j])
        similarity_matrix[i][j] = similarity

    flattened_data['top_similar_personas'] = [[] for _ in range(num_facts)]

    return similarity_matrix

In [150]:
persona_similarity_matrix = calculate_similarity_matrix(flattened_data)

100%|██████████| 1231/1231 [00:04<00:00, 246.25it/s]


In [151]:
n = persona_similarity_matrix.shape[0] 

for i in range(n):
    for j in range(i):
        persona_similarity_matrix[i, j] = persona_similarity_matrix[j, i]

In [186]:
for i in tqdm(range(len(flattened_data))):
    similar_indices = [j for j in range(len(flattened_data)) if persona_similarity_matrix[i][j] > 0.985 and i != j]
    flattened_data.at[i, 'top_similar_personas'] = list(set(flattened_data['persona_id'].iloc[similar_indices]))

100%|██████████| 1231/1231 [00:00<00:00, 4418.08it/s]


In [187]:
unique = []
for i, row in flattened_data.iterrows():
  if row['persona_id'] == 0:
    print('Input: ', row['persona'], '\n')
    for id in row['top_similar_personas']:
      print("\nSimilar: ", flattened_data['persona'][id])
    print('----')

Input:  I am 32.
I do not want a job.
I play video games all day.
I still live at home with my parents.My favorite drink is iced coffee.
I have a black belt in karate.
I m in a jazz band and play the saxophone.
I vacation along lake michigan every summer. 


Similar:  I like jazz.
I like victorian things.
I am a romantic.
I collect dolls.Both my moms tell me they love me when i call them.
I live in chicago.
I have three dogs and two cats.
I like to use a lot of exclamation marks when i type.
Playing quake on slackware linux makes me happy.

Similar:  I am 32.
I do not want a job.
I play video games all day.
I still live at home with my parents.My favorite drink is iced coffee.
I have a black belt in karate.
I m in a jazz band and play the saxophone.
I vacation along lake michigan every summer.

Similar:  I am vegan.
My favorite band is muse.
I work at walmart as a stocker.
My father is a mechanic.I love to chat with my friends.
I like to go shopping with my daughters.
I love kids and d

In [188]:
flattened_data.head()

,persona_id,personas,facts,facts_embeddings,similar_personas,persona,persona_embeddings,top_similar_personas
0,0,[I am 32.\nI do not want a job.\nI play video ...,"I am 32, I do not want a job, I play video gam...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[1, 2, 14, 16, 57, 62, 75, 87, 96, 97, 100, 11...",I am 32.\nI do not want a job.\nI play video g...,"[-0.27412432, -0.19816133, 0.18661429, -0.2283...","[1105, 2, 354, 87]"
1,1,[I am 32.\nI play video games all day.\nI stil...,"I am 32, I play video games all day, I still l...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[0, 1, 2, 11, 14, 16, 27, 57, 62, 70, 75, 76, ...",I am 32.\nI do not want a job.\nI play video g...,"[-0.27595556, -0.19046049, 0.36472562, -0.1551...",[11]
2,2,[I am 32.\nI play video games all day.\nI stil...,"I am 32, I play video games all day, I still l...","[[-0.043408405, -0.039896216, 0.004573693, -0....","[0, 1, 14, 16, 25, 26, 27, 28, 35, 57, 62, 65,...",I am 32.\nI do not want a job.\nI play video g...,"[-0.3358081, -0.24885227, 0.2707594, -0.333563...","[0, 354, 389, 937, 1001]"
3,3,[I write.\nI work at mcdonald s.\nI watch yout...,"I write, I work at mcdonald s, I watch youtube...","[[-0.052719686, -0.04074392, 0.03911773, -0.04...","[35, 39, 41, 45, 92, 95, 98, 99, 158, 159, 160...",I am 32.\nI do not want a job.\nI play video g...,"[-0.34644818, -0.32542628, 0.3416062, -0.24792...","[1162, 1043, 723, 187, 284]"
4,4,[I am bald.\nI like to swim.\nMy favorite drin...,"I am bald, I like to swim, My favorite drink i...","[[-0.024434138, -0.024031263, 0.04923339, -0.0...","[23, 35, 52, 69, 92, 96, 104, 105, 110, 115, 1...",I am 32.\nI do not want a job.\nI play video g...,"[-0.2633983, -0.20231283, 0.28170836, -0.11812...","[481, 1090, 259, 1028, 745, 170, 746, 747, 717..."


In [189]:
flattened_data.to_pickle('FINAL_FINAL_flattened_data.pkl')